In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
#!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
## Remove Users with less than 200 ratings & Books with less than 100 ratings
user_counts = df_ratings['user'].value_counts()
isbn_counts = df_ratings['isbn'].value_counts()

top_reviewers = user_counts[user_counts >= 200].index
popular_isbn = isbn_counts[isbn_counts >= 100].index

df_ratings = df_ratings[df_ratings['user'].isin(top_reviewers)]
df_ratings = df_ratings[df_ratings['isbn'].isin(popular_isbn)]

In [ ]:
## Join ratings & books in one matrix dataframe
df_merged_ratings_with_books = pd.merge(
  right = df_ratings,
  left = df_books,
  on = "isbn"
)

df_merged_ratings_with_books = df_merged_ratings_with_books.drop_duplicates(["title", "user"])

df_titles_by_users = df_merged_ratings_with_books.pivot(
  index = 'title',
  columns = 'user',
  values = 'rating'
).fillna(0)

df_titles_by_users.values.shape

(673, 888)

In [ ]:
# Make model
model = NearestNeighbors(
  n_neighbors=5,
  radius=1.0,
  algorithm="brute",
  leaf_size=30,
  metric="cosine",
  p=2,
  metric_params=None,
  n_jobs=None
)

model.fit(df_titles_by_users.values)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
# function to return recommended books
def get_recommends(book_title=""):
    book_vector = df_titles_by_users.loc[book_title].array.reshape(1, -1)
    distances, indices = model.kneighbors(book_vector, n_neighbors=6)

    recommended_books = []
    for distance, index in zip(distances[0], indices[0]):
        if distance != 0:
            recommended_title = df_titles_by_users.index[index]
            recommended_books.append([recommended_title, distance])

    return [book_title, recommended_books[::-1]]

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016210581447822], ['The Weight of Water', 0.7708583572697412], ['The Surgeon', 0.7699410973804288], ['I Know This Much Is True', 0.7677075092617776], ['The Lovely Bones: A Novel', 0.7234864549790632], ["Where the Heart Is (Oprah's Book Club (Paperback))", 2.220446049250313e-16]]]
You passed the challenge! 🎉🎉🎉🎉🎉
